# Support Vector Machines

- A *support vector machine* (SVM) is a powerful and versatile machine learning model, capable of performing linear or nonlinear classification, regression, and even novelty detection.
- SVMs shine with small to medium-sized nonlinear datasets, especially for classification tasks.
- However, they don't scale very well to large datasets.
- For classification, an SVM separates classes but stays as far away from the closest training instances as possible.
- This is called **large-margin classification**.
- The *support vectors* are the training instances that are located closest to the separating line. 
- The lines that go through these support vectors are make up the "street".

### Soft Margin Classification
- *Hard margin classification* - if we strictly impose that all instances must be off the street and on the correct side.
    - This only works if the data is linearly separable. 
    - This is also very sensitive to outliers.
- To avoid this, we need to find a balance between keeping the street as large as possible and limiting the *margin violations* (i.e. instances that end up in the middle of the street or even on the wrong side). 
- This is called *soft margin classification*.
- In `sklearn` you can set the regularization hyperparameter `C`. 
    - Reducing `C` makes the street larger, but it leads to more violations. (i.e. there is less risk of overfitting).
    - If you reduce it too much, the model will end up underfitting.

In [1]:
# Code below loads the iris dataset and trains a linear SVM classifier to detect iris virginica flowers
from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [2]:
iris = load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = (iris.target == 2) # Iris virginica

In [3]:
svm_clf = make_pipeline(StandardScaler(), LinearSVC(C=1, random_state=42))
svm_clf.fit(X, y)

/Users/tevinachong/Documents/Courses/hands-on-ml/venv/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(C=1, random_state=42))])

In [4]:
X_new = [[5.5, 1.7], [5.0, 1.5]]
svm_clf.predict(X_new)

array([ True, False])

In [5]:
svm_clf.decision_function(X_new)

array([ 0.66163411, -0.22036063])

### Nonlinear SVM Classification
- Some datasets are not linearly separable. 
- One approach to handling these datasets is to add more features, such as polynomial features; in some cases, this can result in a linearly separable dataset. 
- To implement this in `sklearn`, you create a pipeline containing:
    1. `PolynomialFeatures` transformer
    2. `StandardScaler`
    3. `LinearSVC` classifier
- Adding poynomial features can work well with many machine learning algorithms (not just SVMs).
- That being said:
    1. A low polynomial degree cannot deal with very complex datasets.
    2. A high polynomial degree creates a huge number of features, making the model too slow.

In [6]:
from sklearn.datasets import make_moons
from sklearn.preprocessing import PolynomialFeatures

# A toy dataset for binary classification in which the data points are shaped as two interleaving crescent moons
X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

polynomial_svm_clf = make_pipeline(
    PolynomialFeatures(degree=3),
    StandardScaler(),
    LinearSVC(C=10, max_iter=10_000, random_state=42)
)
polynomial_svm_clf.fit(X, y)


/Users/tevinachong/Documents/Courses/hands-on-ml/venv/lib/python3.10/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('standardscaler', StandardScaler()),
                ('linearsvc',
                 LinearSVC(C=10, max_iter=10000, random_state=42))])

### Polynomial Kernel
- When using SVMs, you can apply a mathematical technique called the *kernel trick*.
- The kernel trick makes it possible to get the same result as if you had added many polynomial features, even with a very high degree, without actually having to add them; thus avoiding the combinatorial explosion of the number of features. 

In [7]:
# Implementing the kernel trick with the SVC class
from sklearn.svm import SVC

poly_kernel_svm_clf = make_pipeline(
    StandardScaler(),
    SVC(kernel="poly", degree=3, coef0=1, C=5) #coef0 controls how much the model is influenced by high-degree terms vs low-degree terms
)
poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=5, coef0=1, kernel='poly'))])

### Similarity Features
- Another technique to tackle nonlinear problems is to add features computed using a similarity function, which measures how much each instance resembles a particular *landmark*. 
- You may be wondering how to select the landmarks:
    - The simplest approach is to create a landmark at the location of each and every instance in the dataset. 
    - Doing this creates many dimensions and will thus increase the chances that the transformed training set will be linearly separable. 
    - The downside is that a training set with *m* instances *n* features gets transformed to a training set with *m* instances and *m* features (assuming you drop the original feautres).

    e.g. **Original dataset**
    | Instance | Price ($ USD) | Weight (kg) |
    |--------|--------|--------|
    | 1 | 200 | 300 |
    | 2 | 150 | 200 |
    | 3 | 300 | 150 |

    **Transformed dataset**
    | Instance | Similarity to 1 | Similarity to 2 | Similarity to 3 |
    |--------|--------|--------|--------|
    | 1 | ... | ... | ... |
    | 2 | ... | ... | ... |
    | 3 | ... | ... | ... |

### Gaussian RBF Kernel

- Just like the polynomial features method, the similarity features method can be useful with any ML algorithm, but it may be computationally expensive to compute all the additional features (especially on large training sets).
- Once again, the kernel trick makes it possible to obtain a similar result as if you had added many similarity features, but without actually doing so. 

In [9]:
# Trying the SVC class with the Gaussian RBF kernel
rbf_kernel_svm_clf = make_pipeline(
    StandardScaler(),
    SVC(kernel='rbf', gamma=5, C=0.001)
)
rbf_kernel_svm_clf.fit(X, y)

# Gamma acts like a regularization hyperparameter: if your model is overfitting, reduce gamma. 

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=0.001, gamma=5))])

- Other kernels exist, so how do you decide which one to use?:
    - You should always try the **linear kernel** first. The `LinearSVC` class is much faster than `SVC(kernel="linear")`, especially if the training set is very large.
    - If it is not too large, you should also try kernelized SVMs, starting with the Gaussian RBF kernel; it often works really well.
- The `LinearSVC` class implements an optimized algorithm for linear SVMs. 
    - **It does not support the kernel trick**, but it scales almost linearly with the number of training instances and the number of features. 
    - Its training time complexity is roughly $O(m \times n)$.
    - The algorithm takes longer if you require very high precision (controlled by the tolerance hyperparameter $\epsilon$ or `tol` in `sklearn`).
    - In most classification tasks, the default tolerance is fine. 
- The `SVC` class implements an algorithm that supports the kernel trick.
    - The training time complexity is usually between $O(m^2 \times n)$ and $O(m^3 \times n)$. 
    - This means that it gets dreadfully slow when the number of training instances gets large (e.g. hundreds of thousands of instances).
    - The algorithm is best for small or medium-sized nonlinear training sets.
    - It scales well with the number of features, especially with sparse features (i.e., when each instance has few nonzero features).

| Class | Time Complexity | Out-of-core Support | Scaling Required | Kernel Trick |
|-----|-----|-----|-----|-----|
| LinearSVC | $O(m \times n)$ | No | Yes | No |
| SVC | $O(m^3 \times n)$ | No | Yes | Yes |
| SGDClassifier | $O(m \times n)$ | Yes | No | No |

### SVM Regression
- To use SVMs for regression instead of classification, you need to change the objective:
    - Instead of trying to fit the largest possible street between two classes while limiting margin violations, SVM regression tries to fit as many instances as possible *on* the street while limiting margin violations (i.e., instances *off* the street).
    - The width of the street is controlled by a hyperparameter $\epsilon$.
    - Reducing $\epsilon$ increases the number of support vectors, which regularizes the model. 
- To tackle nonlinear regression tasks, you can use a kernelized SVM model. 

### Under the Hood of Linear SVM Classifiers (ref. page 186)

#### Making Predictions
- A linear SVM classifier predicts the class of a new instance **x** by first computing the decision function $\theta^T \mathbf{x} = \theta_0 x_0 + ... + \theta_n x_n$, where $x_0$ is the bias feature (always equal to 1).
- If the result is positive, then the predicted class $\hat{y}$ is the positive class (1); otherwise it is the negative class (0). *This is exactly like Logistic Regresion*.
- So making predictions is very straightforward. How about training?

#### Training
- This requires finding the weights vector $\mathbf{w}$ and the bias term *b* that make the street, or margin, as wide as possible while limiting the number of margin violations.
- To make the width of the street larger, we need to make $\mathbf{w}$ smaller.
- For example:
    - Let us define the borders of the street as the points where the decision function is equal to -1 or +1.
    - **With a weight of 1:**
        - The decision border ($w_1x_1)$ is -1 at $x_1 = -1$ and it is +1 at $x_1 = 1$**.
        - So the margin is the distance from -1 to 1 **(i.e. 2)**.

    - **With a weight of 0.5:**
        - The decision border $(w_1x_1)$ is -1 at $x_1 = -2$ and it is +1 at $x_1 = 2$.
        - So the margin is the distance from -2 to 2 **(i.e. 4)**
    - Since we want to make the margin as wide as possible, we need to keep $\mathbf{w}$ as small as possible.

- We also want avoid margin violations, so we need the decision function to be greater than 1 for all positive training instances and lower than -1 for all negative training instances. 
- If we define $t^{(i)} = -1$ for negative instances (when $y^{(i)} = 0$) and $t^{(i)} = 1$ for positive instances, then we can write this constraint as $t^{(i)}(\mathbf{w}^T \mathbf{x}^{(i)} + b) \geq 1$ for all instances.
    - *In English*:
        - Once the decision function is positive, t must be positive, once it is negative, t must be negative. Thus always resulting in the multiplication of the two being positive.
    - Therefore, we can express the **hard margin** linear SVM classifier objective function as:
        $$
        \underset{\mathbf{w}, b}{\text{ minimize }}  \frac{1}{2} \mathbf{w}^T\mathbf{w} \\
        
        \text{subject to  } t^{(i)}(\mathbf{w}^T \mathbf{x}^{(i)} + b) \geq 1 \text{ for } i = 1, 2, ... , m
        $$

- To get the soft margin objective, we need to introduce a *slack variable* $\xi^{(i)} \geq 0$ for each instance.
- $\xi^{(i)}$ measures how much the $i^{th}$ instance  is allowed to violate the margin.
- We now have two conflicting objectives: make the slack variable as small as possible to reduce the margin violations, and make $\frac{1}{2} \mathbf{w}^Tw$ as small as possible to increase the margin. This is where the $C$ hyperparameter comes in: it allows us to define the trade-off between these two objectives. 
- This gives us the constrained optimization problem for soft margin classification:
$$
\underset{\mathbf{w}, b, \xi}{\text{minimize  }} \frac{1}{2} \mathbf{w}^T\mathbf{w} + C \sum^m_{i = 1} \xi^{(i)} \\
\text{subject to  } t^{(i)}(\mathbf{w}^T \mathbf{x}^{(i)} + b) \geq 1 - \xi^{(i)} \text{ and } \xi^{(i)} \geq 0  \text{ for } i = 1, 2, ... , m
$$
- The hard margin and soft margin problems are both convex quadratic optimization problems with linear constraints.
    - Such problems are known as *quadratic programming* (QP) problems.
    - Many off-the-shelf solvers are available to solve QP problems using a variety of techniques.
- You can train an SVM using gradient descent to minimize the *hinge loss* or the *squared hinge loss*. 
    - Given an instance $\mathbf{x}$ of the positive class (i.e., with $t=1$), the loss is 0 if the output $s$ of the decision function ($s = \mathbf{w}^Tx+b$) is greater than or equal to 1. *This happens when the instance is off the street and on the positive side*. 
    - Given an instance of the negative class (i.e., with $t=-1$), the loss is 0 if the output $s$ of the decision function is $s \leq -1$. *This happens when the instance is off the street and on the negative side*. 
    - The further away an instance is from the correct side of the margin, the higher the loss: it grows linearly for the hinge loss, and quadratically for the squared hinge loss; making the squared hinge loss more sensitive to outliers. 



### The Dual Problem
- This is another way to train a linear SVM classifier.
- Given a constrained optimization problem, known as the *primal problem*, it is possible to express a different but closely related problem, called its *dual problem*. 


- The dual problem typically gives a lower bound to the solution of the primal problem, but under some conditions it can have the same solution as the primal problem.
- Luckily, the SVM problem happens to meet these conditions, **so you can choose to solve the primal problem or the dual problem;** both will have the same solution. 
- *Dual form of the linear SVM objective*:
$$
\underset{\alpha}{\text{minimize }} \frac{1}{2} \sum^m_{i=1}\sum^m_{j=1} \alpha^{(i)}\alpha^{(j)} t^{(i)}t^{(j)}\mathbf{x}^{(i)\text{T}}\mathbf{x}^{(j)} - \sum^m_{i=1} \alpha^{(i)} \\
\text{subject to } \alpha^{(i)} \geq 0 \text{ for all } i = 1, 2, ..., m \text{ and } \sum^m_{i=1} \alpha^{(i)}t^{(i)} = 0
$$

- Once you find the vector $\hat{\alpha}$ that minimizes this equation (using a QP solver), you can use the equation below to compute the $\hat{\textbf{w}}$ and $\hat{b}$ that minimize the primal problem. In this equation, $n$ represents the number of support vectors.
- *From the dual solution to the primal solution*
$$
\hat{\mathbf{w}} = \sum^m_{i=1} \hat{\alpha}^{(i)}t^{(i)}\mathbf{x}^{(i)} \\
\hat{b} = \frac{1}{n_s} \sum^m_{\underset{\alpha^{(i)}>0}{i=1}} (t^{(i)} - \hat{\mathbf{w}}^\text{T}\mathbf{x}^{(i)})
$$

- The dual problem is faster to solve than the primal one when the number of training instances is smaller than the number of features. 
- More importantly, the dual problem makes the kernel trick possible, while the primal problem does not.

### Kernelized SVMs
- Suppose you want to apply a second-degree polynomial transformation to a two-dimensional training set, then train a linear SVM classifier on the transformed training set. 
- Below is the second-degree polynomial mapping $\phi$ that you want to apply.
$$
\Phi(\mathbf{x}) = 
\begin{pmatrix}
    \begin{pmatrix}
        x_1 \\
        x_2 \\
    \end{pmatrix}
\end{pmatrix}
= 
\begin{pmatrix}
    x_1^2 \\
    \sqrt{2}x_1x_2 \\
    x_2^2 \\
\end{pmatrix}
$$
- Notice that the transformed vector is 3D instead of 2D.

- So let us look at what happens to a couple of 2D vectors, **a** and **b**, if we apply this second-degree polynomial mapping and then compute the dot product of the transformed vectors.
$$
\phi(\mathbf{a})^\text{T} \phi(\mathbf{b}) = 

\begin{pmatrix}
a_1^2 \\
\sqrt{2}a_1a_2 \\
a_2^2
\end{pmatrix}^\text{T}

\begin{pmatrix}
b_1^2 \\
\sqrt{2}b_1b_2 \\
b_2^2
\end{pmatrix}
= a_1^2b_1^2 + 2a_1b_1a_2b_2+a_2^2b_2^2 \\
= (a_1b_1 + a_2b_2)^2 = 

\begin{pmatrix}
    \begin{pmatrix}
    a_1 \\
    a_2
    \end{pmatrix}^\text{T}
    \begin{pmatrix}
    b_1 \\
    b_2
    \end{pmatrix}
\end{pmatrix}^2
= (\mathbf{a}^\text{T}\mathbf{b})^2
$$

- The dot product of the transformed vectors is equal to the sqaure of the dot product of the original vectors. 
- ***Key Insight:***
    - If you apply the transformation $\phi$ to all training instances, then the dual problem will contain the dot product $\phi(\mathbf{x}^{(i)})^\text{T} \phi(\mathbf{x}^{(j)})$
    - But if $\phi$ is the second-degree polynomial transformation defined above, then you can replace this dot product of transformed vectors simply by $(\mathbf{x}^{(i)} \mathbf{x}^{(j)})^2$
    - So you don't need to transform the training instances at all; just replace the dot product by its square.
    - The result will be strictly the same as if you had gone through the trouble of transforming the training set and then fitting a linear SVM algorithm, but the trick makes the whole process much more computationally efficient. 

- The function $K(\mathbf{a}, \mathbf{b}) = (\mathbf{a}^\text{T}\mathbf{b})$^2 is a second-degree polynomial kernel.
- In machine learning, a *kernel* is a function capable of computing the dot product $\phi(\mathbf{a})^\text{T}\phi(\mathbf{b})$, based only on the original vectors **a** and **b**, without having to compute (or even know about) the transformation $\phi$.
- Common kernels:
$$
\textbf{Linear: } K(\mathbf{a}, \mathbf{b}) = \mathbf{a}^\text{T}\mathbf{b} \\
\textbf{Polynomial: } K(\mathbf{a}, \mathbf{b}) = (\gamma\mathbf{a}^\text{T}\mathbf{b} + r)^d \\
\textbf{Gaussian RBF: } K(\mathbf{a}, \mathbf{b}) = \text{exp}(-\gamma || \mathbf{a}-\mathbf{b}||^2) \\
\textbf{Sigmoid: } K(\mathbf{a}, \mathbf{b}) = \text{tanh}(\gamma\mathbf{a}^\text{T}\mathbf{b} + r)
$$